In [ ]:

#     В ЭТОМ КОДЕ:

#     + кластеризуем посты в группе ССММ (про выборы/нет, анонсы/результаты выборов)
#     + посты с анонсами выборов конвертируем в словарики (имя, курс-поток-группа, текст программы)
#     + посты с результатами выборов конвертируем в словарики со всей информацией об итогах выборов
#     + всё это выгружаем в несколько таблиц и дальше работаем с ними в других ноутах

In [2]:
import requests
from bs4 import Tag, NavigableString, BeautifulSoup
import re
from datetime import datetime, date, time

import pandas as pd


In [ ]:
'''
ЧТО ПОФИКСИТЬ:
- БАГ: путает блок текста в шапке репоста с блоком текста в самом репосте (пример: https://vk.com/wall-33327655_2261)
- находить посты с перечнем избранных должностей
- https://vk.com/wall-33327655_5491 - запись удалена, но код этого не видит
- https://vk.com/wall-33327655_5492 - запись не найдена, а код сломался
- https://vk.com/wall-33327655_9414 - код сломался
- были парные кандидаты, найти и расщепить

- ВЫТАЩИТЬ 1-3 СОЗЫВЫ ИЗ ТРЕДОВ
- прогнать 1-3 созывы через сравнение имен
- сделать СКВОЗНУЮ НУМЕРАЦИЮ ssmm_id по мере публикации заявок
'''

# СДЕЛАТЬ ПАРСИНГ ОБСУЖДЕНИЙ В ГРУППЕ:
#     треды с кандидатами 1-3 созывов
#     треды с отчетами о собраний 1-2(3?) созывов


In [567]:
# ссылка на репост без шапки, репостнутая запись - без текста (обновление аватарки):

# req_announcement = requests.get('https://vk.com/wall-33327655_9846')
# html_doc = req_announcement.text
# print(html_doc)

# типы постов:

#     https://vk.com/wall-33327655_2250 -- пост с тегом
#     https://vk.com/wall-33327655_2261 -- пост без тега
#     https://vk.com/wall-33327655_2255 -- post not found
#     https://vk.com/wall-33327655_2262 -- коммент под постом с тегом
#     https://vk.com/wall-33327655_3330 -- репост без шапки поста с адресом 104154624_1
#     https://vk.com/wall-33327655_8489 -- репост с шапкой поста по адресу 77014399_6331


## CREATING A DICTIONARY WITH FULL DATA ABOUT POST/REPLY BY VK-LINK

In [550]:
'''
CREATING A DICTIONARY WITH FULL DATA ABOUT POST/REPLY BY VK-LINK
'''

def vk_post_info_by_link(link_vk, language):
    req_link = requests.get(link_vk)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
#     дефолтные значения
    content_type = 'no text'
    content_subtype = 'other'
    parent_link = ''
    text_lines = []
    link_year_no = 0
    link_month_no = 0
    link_day_no = 0
    link_date = date(1970, 1, 1)
    link_date_time = ''
    link_time = ''
    months_dict = {
        'russian': ['янв', 'фев', 'мар', 'апр', 'мая', 'июн', 'июл', 'авг', 'сен', 'окт', 'ноя', 'дек'],
        'english': ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec'],
#         https://en.wikipedia.org/wiki/Georgian_calendar
        'georgian': ['იან', 'თებ', 'მარ', 'აპრ', 'მაი', 'ივნ', 'ივლ', 'აგვ', 'სექ', 'ოქტ', 'ნოე', 'დეკ']
    }
    months = months_dict[language]

    # ПРОВЕРЯЮ, СУЩЕСТВУЕТ ЛИ ПОСТ
    #     <div class="service_msg service_msg_error">Запись не найдена.</div>
    flg_service_msg_error = soup.find('div', attrs={'class': 'service_msg service_msg_error'})
    if flg_service_msg_error:
        link_type = 'page not found'
    else:
        link_type = 'post'
        # когда ссылка на пост: <link rel="canonical" href="https://vk.com/wall-33327655_2225" />
        # когда ссылка на коммент: <link rel="canonical" href="https://vk.com/wall-33327655_8793?reply=8800" />
        
        # ПРОВЕРЯЮ, НЕ КОММЕНТ ЛИ ЭТО
        rel_canonical = soup.find('link', attrs={'rel': 'canonical'})
        flg_rel_canonical_reply = re.findall('reply=', str(rel_canonical))
#             + ДОБАВИТЬ ПОЛЕ МАТЕРИНСКОЙ ССЫЛКИ
        if flg_rel_canonical_reply:
            link_type = 'comment'
            
        # ПРОВЕРЯЮ, НЕ РЕПОСТ ЛИ ЭТО
        flg_repost = soup.find('div', attrs={'data-copy': True})
        if flg_repost:
            link_type = 'repost'
#             <div class="wall_item single_wall_item bl_item mark_top_verified" data-copy='-77014399_6331'>
            parent_link = 'https://vk.com/wall' + flg_repost['data-copy']
    
    if link_type in ['post','repost']:
        # ВЫТАСКИВАЮ БЛОК С ТЕКСТОМ ПОСТА ПОСТРОЧНО    
        # <div class="wi_body">
        #     <div class="pi_text">

        wi_body = soup.find('div', attrs={'class': 'wi_body'})
        if wi_body is None:
            announcement_list = []
            result_list = []
        else:
            pi_text = wi_body.find('div', attrs={'class': 'pi_text'})
            if pi_text is None:
                announcement_list = []
                result_list = []
            else:
                text_lines = [line for line in pi_text.stripped_strings]
                    # определить тип контента
                    # анонсы: 'Орфография и пунктуация сохранены.'
                    # результаты: 'Таким образом, выборы признаются состоявшимися'
                announcement_list = ['announcement' for line in pi_text.stripped_strings if line.find('Орфография и пунктуация сохранены.') >= 0]
                result_list = ['result' for line in pi_text.stripped_strings if line.find('Таким образом, выборы признаются состоявшимися.') >= 0]
                content_type = 'text'
                if any(election_tag in text_lines for election_tag in ['#выборы@stud_mm','#выборы_ссмм']):
                    content_type = 'election'
    
                if len(announcement_list) > 0:
                    content_subtype = 'announcement'
                elif len(result_list) > 0:
                    content_subtype = 'result'
        
        # ВЫТАСКИВАЮ ДАТУ
        # <div class="wi_info">
        #     <span class="wi_date">
        wi_info = soup.find('div', attrs={'class': 'wi_info'})
        if wi_info:
            wi_date = wi_info.find('span', attrs={'class': 'wi_date'})
            date_list = [date for date in wi_date.stripped_strings]
            link_year_no = int(re.findall(r'[\d]{4}', date_list[0])[0])
            link_month_no = months.index(re.findall(r'[\w]{3}', date_list[0])[0]) + 1
            link_day_no = int((re.findall(r'[\d]+\s', date_list[0])[0]).strip())
            link_date = date(link_year_no, link_month_no, link_day_no)
            link_date_time = date_list[0]
            link_time = re.findall(r'[\d]+\:[\d]+', date_list[0])[0]
        
    if link_type == 'comment':
        reply_link = link_vk[link_vk.index('_')+1:]
        # ВЫТАСКИВАЮ БЛОК С ТЕКСТОМ КОММЕНТА ПОСТРОЧНО    
#             <div class="ReplyItem__body">    180% голосов. Это выборы на мехмате? Это проценты чего?</div>
#             <div class="ReplyItem__date">
#             <a class="item_date" href="https://m.vk.com/wall-33327655_2254?reply=2262#reply2262" rel="noopener">22 &#4316;&#4317;&#4308;&#4315; 2014</a></div>
#         reply_body = soup.find('div', attrs={'class': 'ReplyItem__body'})
        reply_bodies = soup.find_all('div', attrs={'class': 'ReplyItem__body'})
        reply_dates = soup.find_all('a', attrs={'class': 'item_date'})
        reply_links = [ff['href'][ff['href'].index('#reply')+6:] for ff in soup.find_all('a', attrs={'class': 'item_date'})]
        if reply_link in reply_links:
            reply_no = reply_links.index(reply_link)
            content_type = 'text'
            text_lines = [line for line in reply_bodies[reply_no].stripped_strings]
            date_list = [date for date in reply_dates[reply_no].stripped_strings]
            link_year_no = int(re.findall(r'[\d]{4}', date_list[0])[0])
            link_month_no = months.index(re.findall(r'[\w]{3}', date_list[0])[0]) + 1
            link_day_no = int((re.findall(r'[\d]+\s', date_list[0])[0]).strip())
            link_date = date(link_year_no, link_month_no, link_day_no)
        else:
            content_subtype = 'deleted (?)'
            
        parent_link = rel_canonical['href'][: rel_canonical['href'].index('?')]
        
#     баг с переводом строки
    if link_vk == 'https://vk.com/wall-33327655_10007':
        text_lines = text_lines[:5] + [text_lines[5]+' '+text_lines[6]] + text_lines[7:]
    
    dict_link = {}
    dict_link[link_vk] = {
        'link_type': link_type, 
        'content_type': content_type,
        'content_subtype': content_subtype,
        'parent_link': parent_link,
        
        'link_date': link_date, 
        'link_year_no': link_year_no, 
        'link_month_no': link_month_no, 
        'link_day_no': link_day_no, 
        'link_date_time': link_date_time, 
        'link_time': link_time, 
        
        'list_of_lines': text_lines
    }
    
    return dict_link


In [568]:
# тест
# vk_post_info_by_link('https://vk.com/wall-33327655_5106', 'georgian') # -- пост с анонсом +
# vk_post_info_by_link('https://vk.com/wall-33327655_2262', 'georgian') # -- коммент под постом +
# vk_post_info_by_link('https://vk.com/wall-33327655_2263', 'georgian') # -- коммент под постом +
# vk_post_info_by_link('https://vk.com/wall-33327655_3330', 'georgian') # -- репост без шапки +
# vk_post_info_by_link('https://vk.com/wall-33327655_2261', 'georgian') # -- репост без шапки - ПОЧЕМУ ТУТ text, а не no text
# vk_post_info_by_link('https://vk.com/wall-33327655_8489', 'georgian') # -- репост с шапкой +
# vk_post_info_by_link('https://vk.com/wall-33327655_9846', 'georgian') # -- удаленный коммент (?)
# vk_post_info_by_link('https://vk.com/wall-33327655_2225', 'georgian') # -- пост с кандидатами в КОММЕНТЕ 2232
# vk_post_info_by_link('https://vk.com/wall-33327655_2232', 'georgian') # -- коммент с кандидатами

# vk_post_info_by_link('https://vk.com/wall-33327655_10007', 'georgian') # -- РАЗНОЕ
# https://vk.com/wall-33327655_10007



## functions for processing of posts about elections

### functions for posts with announcements

In [32]:

# вытаскивает из строки номера курса и потока (в предположении, что они в этой строке есть)

# для обычных потоков
def year_stream_no_by_line(line):
    if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', line):
        year_stream_line = (re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', line))[0]
        year_stream_line_d = re.sub('\D', '', year_stream_line)
        year_no = int(year_stream_line_d[0])
        stream_no = int(year_stream_line_d[1])
    elif re.findall(r'\D(\d\.\d)\D', line):
        year_stream_line = (re.findall(r'\D(\d\.\d)\D', line))[0]
        year_no = year_stream_line[0]
        stream_no = year_stream_line[2]
    
    year_stream_no = str(year_no) + '.' + str(stream_no)
    return year_stream_no, year_no, stream_no

# для магистратуры
def magister_year_by_line(line):
#     если номер года записан цифрами
    if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', line):
        year_stream_line = (re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', line))[0]
        year_no = int(year_stream_line[1])
        year_stream_no = 'm' + str(year_no)
#     если номер года записан буквами
    elif re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', line):
        year_stream_line = (re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', line))[0]
        year_no = 0
        if str.lower(year_stream_line[1]) == 'перв':
            year_no = 1
        elif str.lower(year_stream_line[1]) == 'втор':
            year_no = 2
        year_stream_no = 'm' + str(year_no)
    return year_stream_no, year_no

# примеры для магистратуры:
# МАГИСТРЫ ПЕРВОГО ГОДА ОБУЧЕНИЯ (выборы завтра на второй паре в аудитории 12-03) +
# МАГИСТРЫ, 2 ГОД ОБУЧЕНИЯ +
# МАГИСТРЫ 2 ГОДА ОБУЧЕНИЯ +
# МАГИСТРЫ 1-ГО ГОДА ОБУЧЕНИЯ +
# МАГИСТРЫ, 1 КУРС +

# line = 'МАГИСТРЫ, 1 КУРС'
# magister_year_by_line(line)

In [569]:
# line1 = '— ЯНКОВА Маргарита Михайловна' +
# line1 = '— ОСТРОУХОВА Наталья Владимировна, группа 402' 
# line1 = '— ЧЕБОТАЕВА Виктория Алексеевна, 332 группа' +
# line1 = '— ПРОМЫСЛОВ Платон Валерьевич, гр.409' +
# line1 = '— БУН-ЯН-ТУН Владислав Владимирович, группа 125' +
# line1 = 'Перейдём к старшим курсам. Есть две новости, хорошая и плохая. Плохая: от механиков 5 курса до сих пор не поступило ни одной заявки. Поторопитесь, не оставляйте свой поток без представителя — время до полуночи ещё есть! Хорошая: на экономпотоке четверокурсников кандидат в наличии, с ним и знакомим.'

# # r'\—([\w\s\-]+)[\,\.\s\w]*([\d]*)'

# if len(re.findall(r'\—([\w\s\-]+)', line1)) > 0 and re.search(r'\—([\w\s\-]+)', line1).start() == 0:
#     name_line = (re.findall(r'\—([\w\s\-]+)', line1))
#     print(name_line)
#     name = (re.sub('\W\s', '', name_line[0])).strip().title()
#     if re.sub('\D', '', line1):
#         academic_group_no = int(re.sub('\D', '', line1))
#         print(academic_group_no)

# print(name)

In [517]:
# СЛОВАРЬ ИНФОРМАЦИИ ПРО КАНДИДАТОВ (по списку строк с текстом)

def nomination_dict_by_name_line(nominated_dict, nomination_pattern, line, i):
        if (
            len(re.findall(nomination_pattern, line)) > 0  # если в строке есть такой паттерн
            and re.search(nomination_pattern, line).start() == 0  # с начала строки
            and len(re.findall(nomination_pattern, line)[0].strip().split(' ')) <= 6 # не более чем из 6 слов
        ):
            name_line_no = i
            name_line = re.findall(nomination_pattern, line)
            name = (re.sub('\W\s', '', name_line[0])).strip().title()
            if re.sub('\D', '', line):
                academic_group_no = int(re.sub('\D', '', line))
            else:
                academic_group_no = ''
            another_one_dict = {}
            another_one_dict[name] = {
                'academic_group_no': academic_group_no, 
                'name_line_no': name_line_no
            }
            nominated_dict = {**nominated_dict, **another_one_dict}
        return nominated_dict

# по списку строк с текстом составляем СЛОВАРЬ: имя: курс-поток, группа, номер первой строки в тексте
def stream_nomination_by_list(list_of_lines):
    nominated_dict = {}
    bug_patterns = [r'(ШИШКИН Игнат Владимирович), группа 122', r'(УСТИНОВ Никита Евгеньевич), группа 310.']
    
    for i in range(len(list_of_lines)):
#         В НАЧАЛЕ ШАБЛОНА БЫЛО \s (убрал, так как иногда забывают пробел, а алгоритм всё равно жадный)
#         r'[\w\s]+,\sгруппа\s[\d]{3}' - для фиксированного написания группы (и не для маги)
        nomination_pattern = r'[\—\–]([\w\s\-]+)'  # на первой позиции два тире: длинный или средний
        nominated_dict = nomination_dict_by_name_line(nominated_dict, nomination_pattern, list_of_lines[i], i)
        
        for nomination_pattern in bug_patterns:
            if (
                len(re.findall(nomination_pattern, list_of_lines[i])) > 0  # если в строке есть такой паттерн
                and re.search(nomination_pattern, list_of_lines[i]).start() == 0  # с начала строки
                and len(re.findall(nomination_pattern, list_of_lines[i])[0].strip().split(' ')) <= 6 # не более чем из 6 слов
            ):
                nominated_dict = nomination_dict_by_name_line(nominated_dict, nomination_pattern, list_of_lines[i], i)
                
    # первый и последний индексы программы кандидата в списке
    name_list = list(nominated_dict.keys())
    start_index_list = [nominated_dict[key]['name_line_no'] for key in list(nominated_dict.keys())]
    last_index_list = [i-1 for i in start_index_list if start_index_list.index(i) > 0]
    last_index_list.append(len(list_of_lines)-1)
    
    dict_of_nominated_upd = {}
    
    for name in name_list:
        dict_of_nominated_upd[name] = [start_index_list[name_list.index(name)], last_index_list[name_list.index(name)]]
    
    # обновить словарь: имя: номер группы, текст программы
    for name in dict_of_nominated_upd:
        program_lines = []

        for k in range(dict_of_nominated_upd[name][0]+1, dict_of_nominated_upd[name][1]+1):
            program_lines.append(list_of_lines[k])
        program_txt = '\n'.join(program_lines)

        nominated_dict[name].update({'program_txt': program_txt})
        del nominated_dict[name]['name_line_no']

    return nominated_dict


# вся инфа про номинации на потоке
def stream_nomination_info_by_dict(list_of_lines):
    year_stream_no = ''
    year_no, stream_no = 0, 0
    
    for i in range(len(list_of_lines)):
        if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]):
            year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
        if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', list_of_lines[i]) or re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', list_of_lines[i]):
            year_stream_no, year_no = magister_year_by_line(list_of_lines[i])
                
    dict_nominations = stream_nomination_by_list(list_of_lines)
    
    dict_stream_elections = {
        'year_stream_no': year_stream_no,
        'year_no': year_no, 
        'stream_no': stream_no, 
        'dict_nominations': dict_nominations
    }
    
    return dict_stream_elections



### functions for posts with results

In [352]:
# СЛОВАРЬ ИНФОРМАЦИИ ПРО РЕЗУЛЬТАТЫ ВЫБОРОВ (по списку строк с текстом)

# ДОБАВИТЬ МАГИСТРАТУРУ

# по списку строк с кандидатами составляем СЛОВАРЬ: имя: голоса, статус
def stream_election_result_by_list(nominated_with_votes):
    dict_nominated = {}

    for line in nominated_with_votes:
        kek = (re.findall(r'[\—\s]*([\w\-\s]+)[\:\;]\s([\d]+)\sголос', line))[0]
        name = kek[0].strip().title()
        votes_cnt = kek[1]
        dict_nominated[name] = {'votes_cnt': votes_cnt}
        if (re.findall(r' избран', line)) and not (re.findall(r' не избран', line)):
            elected_flg = 1
        else:
            elected_flg = 0
        dict_nominated[name].update({'elected_flg': elected_flg})
    return dict_nominated


# вся инфа про результаты выборов на потоке
def stream_election_info_by_dict(list_of_lines):
    nominated_with_votes = []
    year_stream_no = ''
    year_no, stream_no = 0, 0
    voted_cnt, stream_size, invalid_cnt, NOTA_cnt = 0, 0, 0, 0
    
    for i in range(len(list_of_lines)):
                    
        if re.findall(r'\d\s[\w]{4},\s\d\s[\w]{5}', list_of_lines[i]) or re.findall(r'\D(\d\.\d)\D', list_of_lines[i]):
            year_stream_no, year_no, stream_no = year_stream_no_by_line(list_of_lines[i])
        if re.findall(r'(МАГИСТРЫ)[\s,]{1,2}([\d])', list_of_lines[i]) or re.findall(r'(МАГИСТРЫ)[\W]+([\w]{4})', list_of_lines[i]):
            year_stream_no, year_no = magister_year_by_line(list_of_lines[i])
            
        if re.findall('Явка: ', list_of_lines[i]):
            turnout_line = (re.findall(r'[\d]+\s[\w]+\sиз\s[\d]+', list_of_lines[i]))[0]
            voted_cnt = int(((re.findall(r'[\d]+\s', turnout_line))[0]).strip())
            stream_size = int(((re.findall(r'\s[\d]+', turnout_line))[0]).strip())
            
        if len(re.findall(r'\s[\d]+\sнедействительн', list_of_lines[i])) == 0: # отдельно, т.к. может такой записи не быть
            pass
        else:
            invalid_line_no = i
            invalid_line = (re.findall(r'\s[\d]+\sнедействительн', list_of_lines[i]))[0]
            invalid_cnt = re.sub('\D', '', invalid_line)
            
        if re.findall('Против всех', list_of_lines[i]):
            NOTA_line_no = i
            NOTA_line = (re.findall(r'\s[\d]+\sголос', list_of_lines[i]))[0]
            NOTA_cnt = re.sub('\D', '', NOTA_line)
            
        if re.findall('[\—\s]*([\w\-\s]+)[\:\;]\s([\d]+)\sголос', list_of_lines[i]) and not (re.findall('Против всех', list_of_lines[i])):
            nominated_with_votes.append(list_of_lines[i])
            
        if re.findall('Подтверждено ответственн', list_of_lines[i]):
            organizer_line = (re.findall(r'за\sвыборы\s[\w\s,]+', list_of_lines[i]))[0]
            organizer_line = organizer_line[10::]
            organizer_list = re.split(', | и ',organizer_line)
    
    dict_nominated_with_votes = stream_election_result_by_list(nominated_with_votes)
    
    dict_stream_elections = {
        'year_stream_no': year_stream_no,
        'year_no': year_no, 
        'stream_no': stream_no, 
        'stream_size': stream_size, 
        'voted_cnt': voted_cnt,
        'invalid_cnt': invalid_cnt,
        'NOTA_cnt': NOTA_cnt,
        'organizers': organizer_list,
        'dict_nominated_with_votes': dict_nominated_with_votes
    }
    
    return dict_stream_elections


In [556]:
def fix_main_dict(test_dict):
#     поправка из-за коммента с доп.кандидатами
    if 'https://vk.com/wall-33327655_2232' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_2232']['content_type'] = 'election'
        test_dict['https://vk.com/wall-33327655_2232']['content_subtype'] = 'announcement'
#     поправка из-за коммента с доп.кандидатами
    if 'https://vk.com/wall-33327655_2305' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_2305']['content_type'] = 'election'
        test_dict['https://vk.com/wall-33327655_2305']['content_subtype'] = 'announcement'
#     поправка из-за коммента с доп.кандидатами
    if 'https://vk.com/wall-33327655_7650' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_7650']['content_type'] = 'election'
        test_dict['https://vk.com/wall-33327655_7650']['content_subtype'] = 'announcement'
        
#     поправка из-за отсутствия кодовой фразы
    if 'https://vk.com/wall-33327655_2230' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_2230']['content_subtype'] = 'announcement'
#     поправка из-за отсутствия выборного хэштега
    if 'https://vk.com/wall-33327655_7509' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_7509']['content_type'] = 'election'
        test_dict['https://vk.com/wall-33327655_7509']['content_subtype'] = 'result'
#     поправка из-за отсутствия кодовой фразы
    if 'https://vk.com/wall-33327655_7657' in test_dict.keys():
        test_dict['https://vk.com/wall-33327655_7657']['content_type'] = 'election'
        test_dict['https://vk.com/wall-33327655_7657']['content_subtype'] = 'announcement'

    return test_dict

def fix_announcement_links(announcement_links):
# поправки из-за единичных нестандартных случаев
    if 'https://vk.com/wall-33327655_2232' in announcement_links.keys():
        announcement_links['https://vk.com/wall-33327655_2232']['nominated_info']['year_stream_no'] = '1.2'
        announcement_links['https://vk.com/wall-33327655_2232']['nominated_info']['year_no'] = 1
        announcement_links['https://vk.com/wall-33327655_2232']['nominated_info']['stream_no'] = 2
    if 'https://vk.com/wall-33327655_2305' in announcement_links.keys():
        announcement_links['https://vk.com/wall-33327655_2305']['nominated_info']['year_stream_no'] = '3.1'
        announcement_links['https://vk.com/wall-33327655_2305']['nominated_info']['year_no'] = 3
        announcement_links['https://vk.com/wall-33327655_2305']['nominated_info']['stream_no'] = 1
    if 'https://vk.com/wall-33327655_7650' in announcement_links.keys():
        announcement_links['https://vk.com/wall-33327655_7650']['nominated_info']['year_stream_no'] = '1.3'
        announcement_links['https://vk.com/wall-33327655_7650']['nominated_info']['year_no'] = 1
        announcement_links['https://vk.com/wall-33327655_7650']['nominated_info']['stream_no'] = 3
    return announcement_links

def fix_result_links(result_links):
    if 'https://vk.com/wall-33327655_7695' in result_links.keys():
        result_links['https://vk.com/wall-33327655_7695']['stream_election_info']['year_stream_no'] = 'm1'
        result_links['https://vk.com/wall-33327655_7695']['stream_election_info']['year_no'] = 1
    if 'https://vk.com/wall-33327655_9205' in result_links.keys():
        result_links['https://vk.com/wall-33327655_9205']['stream_election_info']['year_stream_no'] = '6.3'
        result_links['https://vk.com/wall-33327655_9205']['stream_election_info']['year_no'] = 6
        result_links['https://vk.com/wall-33327655_9205']['stream_election_info']['stream_no'] = 3
    if 'https://vk.com/wall-33327655_9267' in result_links.keys():
        result_links['https://vk.com/wall-33327655_9267']['stream_election_info']['year_stream_no'] = '6.1'
        result_links['https://vk.com/wall-33327655_9267']['stream_election_info']['year_no'] = 6
        result_links['https://vk.com/wall-33327655_9267']['stream_election_info']['stream_no'] = 1
    return result_links


In [619]:
# словарик для ручного исправления имен кандидатов

def fix_name(name):
    # incorrect: correct
    correct_names_dict = {
    #     из-за Е/Ё:
        'Муравьева Анна Сергеевна': 'Муравьёва Анна Сергеевна',
        'Третьяков Артем Алексеевич': 'Третьяков Артём Алексеевич',
        'Рогачев Александр Викторович': 'Рогачёв Александр Викторович',
        'Третьяков Петр Максимович': 'Третьяков Пётр Максимович',
        'Пушкарева Анна Константиновна': 'Пушкарёва Анна Константиновна',
    #     из-за других букв:
        'Порфирьева Наталья Алексеевна': 'Порфирьева Наталия Алексеевна',
        'Лобанова Елена Валерьевна': 'Лобанова Алёна Валерьевна',
        'Иллизаров Год Эмануилович': 'Илизаров Год Эмануилович',
        'Гусейнов Сеймур Элданизович': 'Гусейнов Сэймур Элданизович',
    #     из-за пропущенного отчества:
        'Лапина София': 'Лапина Софья Михайловна',   
    #     из-за перепутанного отчества:
        'Сакаев Роберт Викторович': 'Сакаев Роберт Марселевич',   
    }
    if name in correct_names_dict.keys():
        return correct_names_dict[name]
    else:
        return name

fix_name('Иллизаров Год Эмануилович')

'Илизаров Год Эмануилович'

## Processing of all election posts for 4-11 convocations

### launching

In [620]:
# ГЕНЕРАТОР СЛОВАРЯ ДЛЯ ИНТЕРВАЛА ССЫЛОК НА ПОСТЫ С АНОНСАМИ ВЫБОРОВ
# ГЕНЕРАТОР СЛОВАРЯ ДЛЯ ИНТЕРВАЛА ССЫЛОК НА ПОСТЫ С РЕЗУЛЬТАТАМИ ВЫБОРОВ

from itertools import chain

announcement_links = {}
result_links = {}

for num in chain(
#     range(8330, 8331) #, # test
    
    range(2200, 2360) , # созыв 4 +
#     range(3140, 3530) #, # созыв 5 +
#     range(5040, 5360) #, # созыв 6 +
#     range(6540, 6720) #, # созыв 7 +
#     range(7470, 7780), # созыв 8 +
#     range(8240, 8420), # созыв 9 +
#     range(9100, 9400), # созыв 10 +
#     range(9800, 10040) # созыв 11 +
):
    link = 'https://vk.com/wall-33327655_' + str(num)

    test_dict = vk_post_info_by_link(link, 'georgian')
    test_dict = fix_main_dict(test_dict)
    
    print('{0} : {1} \t\t/ {2} \t\t/ {3}'.format(link, test_dict[link]['link_type'], test_dict[link]['content_type'], test_dict[link]['content_subtype']))

    for d in test_dict.values():
        if d['content_type'] == 'election' and d['content_subtype'] == 'announcement':
            announcement_links = {**announcement_links, **test_dict}
        if d['content_type'] == 'election' and d['content_subtype'] == 'result':
            result_links = {**result_links, **test_dict}


for link in announcement_links:
    announcement_links[link].update({'nominated_info': stream_nomination_info_by_dict(announcement_links[link]['list_of_lines'])})
    del announcement_links[link]['list_of_lines']
    
for link in result_links:
    result_links[link].update({'stream_election_info': stream_election_info_by_dict(result_links[link]['list_of_lines'])})
    del result_links[link]['list_of_lines']

# поправки из-за единичных нестандартных случаев
announcement_links = fix_announcement_links(announcement_links)
result_links = fix_result_links(result_links)

# print(announcement_links)
# print(result_links)


https://vk.com/wall-33327655_2200 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2201 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2202 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2203 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2204 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2205 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2206 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2207 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2208 : post 		/ election 		/ other
https://vk.com/wall-33327655_2209 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2210 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2211 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2212 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_2213 : post 		/ election 		/ announcement
https://vk.com/wall-33327655_

https://vk.com/wall-33327655_2323 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2324 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2325 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2326 : post 		/ election 		/ result
https://vk.com/wall-33327655_2327 : page not found 		/ no text 		/ other
https://vk.com/wall-33327655_2328 : post 		/ election 		/ result
https://vk.com/wall-33327655_2329 : post 		/ election 		/ result
https://vk.com/wall-33327655_2330 : post 		/ election 		/ result
https://vk.com/wall-33327655_2331 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2332 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2333 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2334 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2335 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2336 : comment 		/ text 		/ other
https://vk.com/wall-33327655_2337 : comment 		/ text 		/ other
https:/

In [525]:
# announcement_links['https://vk.com/wall-33327655_7650']

### creating tables about candidates and streams

In [515]:
# БОЛЬШАЯ ТАБЛИЦА ПО ПОТОКАМ

announcement_streams = {}
for link in announcement_links.keys():
    announcement_streams[link] = {
        'year_x_year_stream': str(announcement_links[link]['link_year_no']) + '_' + announcement_links[link]['nominated_info']['year_stream_no'],
        'link_date': announcement_links[link]['link_date'],
        'link_year_no': announcement_links[link]['link_year_no'], 
        'link_month_no': announcement_links[link]['link_month_no'], 
        'link_day_no': announcement_links[link]['link_day_no'], 
        'link_time': announcement_links[link]['link_time'], 
        'year_stream_no': announcement_links[link]['nominated_info']['year_stream_no'], 
        'year_no': announcement_links[link]['nominated_info']['year_no'], 
        'stream_no': announcement_links[link]['nominated_info']['stream_no']
    }
# announcement_streams

announcement_streams_df =  pd.DataFrame(announcement_streams.items(), columns=['link', 'params'])
announcement_streams_df_2 = pd.concat([announcement_streams_df.drop(['params'], axis=1), announcement_streams_df['params'].apply(pd.Series)], axis=1)
announcement_streams_df_3 = announcement_streams_df_2.sort_values(['year_stream_no', 'link'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)
announcement_streams_df_3


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no
0,https://vk.com/wall-33327655_7621,2018_1.1,2018-10-23,2018,10,23,0:00,1.1,1,1
1,https://vk.com/wall-33327655_7479,2018_1.2,2018-10-12,2018,10,12,21:45,1.2,1,2
2,https://vk.com/wall-33327655_7645,2018_1.3,2018-10-23,2018,10,23,23:38,1.3,1,3
3,https://vk.com/wall-33327655_7650,2018_1.3,2018-10-24,2018,10,24,,1.3,1,3
4,https://vk.com/wall-33327655_7622,2018_2.1,2018-10-23,2018,10,23,0:03,2.1,2,1
5,https://vk.com/wall-33327655_7644,2018_2.3,2018-10-23,2018,10,23,23:19,2.3,2,3
6,https://vk.com/wall-33327655_7619,2018_3.1,2018-10-21,2018,10,21,23:45,3.1,3,1
7,https://vk.com/wall-33327655_7657,2018_3.2,2018-10-25,2018,10,25,0:10,3.2,3,2
8,https://vk.com/wall-33327655_7552,2018_3.3,2018-10-17,2018,10,17,23:27,3.3,3,3
9,https://vk.com/wall-33327655_7623,2018_3.4,2018-10-23,2018,10,23,0:06,3.4,3,4


In [510]:
announcement_streams_df_3.to_csv('ssmm_announcement_info_conv8.csv', sep='\t', encoding='utf-8', index=False)

In [ ]:
'''
(ИСПРАВИЛ - вот и пригодился ключ)
!!!БАГ!!! 
РАБОТАЕТ ТОЛЬКО ПО ОТДЕЛЬНЫМ СОЗЫВОМ!!! 
ИНАЧЕ В СЛОВАРИКЕ ПЕРЕЗАПИСЫВАЕТСЯ ИНФА ПО КЛЮЧУ-ИМЕНИ
ДЛЯ КАНДИДАТОВ С НЕСКОЛЬКИМИ СОЗЫВАМИ
'''

In [621]:
# БОЛЬШАЯ ТАБЛИЦА ПО КАНДИДАТАМ

announcement_names = {}
for link in announcement_links.keys():
    for name in announcement_links[link]['nominated_info']['dict_nominations'].keys():
        announcement_names[str(announcement_links[link]['link_year_no']) + "_" + "_".join(fix_name(name).split())] = {
            'name': fix_name(name),
            'link_year_no': announcement_links[link]['link_year_no'], 
            'year_stream_no': announcement_links[link]['nominated_info']['year_stream_no'], 
            'year_no': announcement_links[link]['nominated_info']['year_no'], 
            'stream_no': announcement_links[link]['nominated_info']['stream_no'], 
            'academic_group_no': announcement_links[link]['nominated_info']['dict_nominations'][name]['academic_group_no'], 
            'program_txt': announcement_links[link]['nominated_info']['dict_nominations'][name]['program_txt'],
            'link': link
        }
# поправка из-за парной заявки
# if 'Шишкин Игнат Владимирович' in announcement_names.keys() and 'Винокурова Алина Валерьевна' in announcement_names.keys() and announcement_names['Шишкин Игнат Владимирович']['link_year_no'] == 2019:
#     announcement_names['Шишкин Игнат Владимирович']['program_txt'] = announcement_names['Винокурова Алина Валерьевна']['program_txt']
if '2019_Шишкин_Игнат_Владимирович' in announcement_names.keys() and '2019_Винокурова_Алина_Валерьевна' in announcement_names.keys():
    announcement_names['2019_Шишкин_Игнат_Владимирович']['program_txt'] = announcement_names['2019_Винокурова_Алина_Валерьевна']['program_txt']

announcement_names_df =  pd.DataFrame(announcement_names.items(), columns=['link_year_no_x_name', 'params'])
announcement_names_df_2 = pd.concat([announcement_names_df.drop(['params'], axis=1), announcement_names_df['params'].apply(pd.Series)], axis=1)
announcement_names_df_2[0:60]
# .head(10)
# announcement_names


,link_year_no_x_name,name,link_year_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link
0,2014_Суркова_Анастасия_Юрьевна,Суркова Анастасия Юрьевна,2014,2.2,2,2,209,"«Подавая заявку в прошлом году в студсовет, яв...",https://vk.com/wall-33327655_2202
1,2014_Денисов_Илья_Олегович,Денисов Илья Олегович,2014,2.2,2,2,207,"«Вечер добрый, коллеги!\nЯ бы хотел подать зая...",https://vk.com/wall-33327655_2202
2,2014_Тюрина_Татьяна_Игоревна,Тюрина Татьяна Игоревна,2014,2.2,2,2,209,«Уже во второй раз избираюсь в студенческий со...,https://vk.com/wall-33327655_2202
3,2014_Дорман_Елена_Вячеславовна,Дорман Елена Вячеславовна,2014,2.3,2,3,226,"«По моему мнению, за прошедший год я не плохо ...",https://vk.com/wall-33327655_2203
4,2014_Оськин_Александр_Андреевич,Оськин Александр Андреевич,2014,2.3,2,3,225,«Прежде всего здравствуйте:)\nНа протяжении по...,https://vk.com/wall-33327655_2203
5,2014_Хапкин_Артём_Владимирович,Хапкин Артём Владимирович,2014,4.1,4,1,402,"«Студент 4ого курса кафедры алгебры, избираюсь...",https://vk.com/wall-33327655_2210
6,2014_Тулякова_Анна_Владимировна,Тулякова Анна Владимировна,2014,4.1,4,1,403,"«Состояла в ССММ в 2012-2013, работала в групп...",https://vk.com/wall-33327655_2210
7,2014_Матвеев_Дмитрий_Александрович,Матвеев Дмитрий Александрович,2014,4.1,4,1,402,"«Много лет работаю в Студсовете, координатор г...",https://vk.com/wall-33327655_2210
8,2014_Бунеев_Артём_Валерьевич,Бунеев Артём Валерьевич,2014,4.2,4,2,410,не приложил к заявке рассказ о себе,https://vk.com/wall-33327655_2211
9,2014_Борисов_Василий_Александрович,Борисов Василий Александрович,2014,4.2,4,2,410,"«Избираюсь второй раз, за прошедший год участв...",https://vk.com/wall-33327655_2211


In [622]:
announcement_names_df_2.to_csv('ssmm_candidates_info_conv4.csv', sep='\t', encoding='utf-8', index=False)

 ### creating tables about results

In [220]:
# БОЛЬШАЯ ТАБЛИЦА ПО ПОТОКАМ

result_streams = {}
for link in result_links.keys():
    result_streams[link] = {
        'year_x_year_stream': str(result_links[link]['link_year_no']) + '_' + result_links[link]['stream_election_info']['year_stream_no'],
        'link_date': result_links[link]['link_date'],
        'link_year_no': result_links[link]['link_year_no'], 
        'link_month_no': result_links[link]['link_month_no'], 
        'link_day_no': result_links[link]['link_day_no'], 
        'link_time': result_links[link]['link_time'], 
        'year_stream_no': result_links[link]['stream_election_info']['year_stream_no'], 
        'year_no': result_links[link]['stream_election_info']['year_no'], 
        'stream_no': result_links[link]['stream_election_info']['stream_no'], 
        'stream_size': result_links[link]['stream_election_info']['stream_size'], 
        'voted_cnt': result_links[link]['stream_election_info']['voted_cnt'], 
        'invalid_cnt': result_links[link]['stream_election_info']['invalid_cnt'], 
        'NOTA_cnt': result_links[link]['stream_election_info']['NOTA_cnt'], 
        'organizers': result_links[link]['stream_election_info']['organizers']
    }
# announcement_streams

result_streams_df =  pd.DataFrame(result_streams.items(), columns=['link', 'params'])
result_streams_df_2 = pd.concat([result_streams_df.drop(['params'], axis=1), result_streams_df['params'].apply(pd.Series)], axis=1)
result_streams_df_3 = result_streams_df_2.sort_values(['year_stream_no', 'link'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)
result_streams_df_3


,link,year_x_year_stream,link_date,link_year_no,link_month_no,link_day_no,link_time,year_stream_no,year_no,stream_no,stream_size,voted_cnt,invalid_cnt,NOTA_cnt,organizers
0,https://vk.com/wall-33327655_7676,2018_1.1,2018-10-26,2018,10,26,17:00,1.1,1,1,168,117,7,4,"[Георгием Малиновским, Олегом Бузенковым]"
1,https://vk.com/wall-33327655_7509,2018_1.2,2018-10-14,2018,10,14,21:00,1.2,1,2,151,104,2,8,"[Георгием Малиновским, Кириллом Филатовым]"
2,https://vk.com/wall-33327655_7685,2018_1.3,2018-10-27,2018,10,27,14:00,1.3,1,3,123,98,3,5,"[Сенотовым Валерием, Малиновским Георгием]"
3,https://vk.com/wall-33327655_7677,2018_2.1,2018-10-26,2018,10,26,19:00,2.1,2,1,134,81,5,1,"[Валерией Фиткович, Николаем Малаховым]"
4,https://vk.com/wall-33327655_7684,2018_2.3,2018-10-27,2018,10,27,12:00,2.3,2,3,107,75,1,4,"[Сенотовым Валерием, Малиновским Георгием]"
5,https://vk.com/wall-33327655_7675,2018_3.1,2018-10-26,2018,10,26,14:00,3.1,3,1,107,58,3,11,"[Валерием Сенотовым, Александром Тонконоговым]"
6,https://vk.com/wall-33327655_7728,2018_3.2,2018-11-12,2018,11,12,22:56,3.2,3,2,94,49,1,4,"[Ильёй Денисовым, Дарьей Волковой]"
7,https://vk.com/wall-33327655_7670,2018_3.3,2018-10-25,2018,10,25,23:52,3.3,3,3,103,55,0,8,"[Валерием Сенотовым, Александром Тонконоговым]"
8,https://vk.com/wall-33327655_7681,2018_3.4,2018-10-27,2018,10,27,10:00,3.4,3,4,40,25,0,0,"[Валерием Сенотовым, Александром Тонконоговым]"
9,https://vk.com/wall-33327655_7694,2018_4.1,2018-11-02,2018,11,2,22:25,4.1,4,1,86,45,2,11,"[Денисовым Ильей, Воробьевой Наталией]"


In [221]:
result_streams_df_3.to_csv('ssmm_results_info_conv8.csv', sep='\t', encoding='utf-8', index=False)

In [222]:
# ОТДЕЛЬНАЯ ТАБЛИЦА ПО ОРГАНИЗАТОРАМ

organizers_df = result_streams_df_3[['year_x_year_stream','organizers']].apply(lambda x: x.explode()).reset_index().drop(['index'], axis=1)
# organizers_df


In [223]:
organizers_df.to_csv('ssmm_organizers_info_conv8.csv', sep='\t', encoding='utf-8', index=False)

In [623]:
# БОЛЬШАЯ ТАБЛИЦА ПО КАНДИДАТАМ

result_names = {}
for link in result_links.keys():
    for name in result_links[link]['stream_election_info']['dict_nominated_with_votes'].keys():
        result_names[str(result_links[link]['link_year_no']) + "_" + "_".join(fix_name(name).split())] = {
            'name': fix_name(name),
            'link_year_no': result_links[link]['link_year_no'], 
            'year_stream_no': result_links[link]['stream_election_info']['year_stream_no'], 
            'year_no': result_links[link]['stream_election_info']['year_no'], 
            'stream_no': result_links[link]['stream_election_info']['stream_no'], 
            'votes_cnt': result_links[link]['stream_election_info']['dict_nominated_with_votes'][name]['votes_cnt'],
            'elected_flg': result_links[link]['stream_election_info']['dict_nominated_with_votes'][name]['elected_flg'],
            'link': link
        }

result_names_df =  pd.DataFrame(result_names.items(), columns=['link_year_no_x_name', 'params'])
result_names_df_2 = pd.concat([result_names_df.drop(['params'], axis=1), result_names_df['params'].apply(pd.Series)], axis=1)
result_names_df_2
# .head(10)


,link_year_no_x_name,name,link_year_no,year_stream_no,year_no,stream_no,votes_cnt,elected_flg,link
0,2014_Суркова_Анастасия_Юрьевна,Суркова Анастасия Юрьевна,2014,2.2,2,2,53,1,https://vk.com/wall-33327655_2215
1,2014_Денисов_Илья_Олегович,Денисов Илья Олегович,2014,2.2,2,2,61,1,https://vk.com/wall-33327655_2215
2,2014_Тюрина_Татьяна_Игоревна,Тюрина Татьяна Игоревна,2014,2.2,2,2,48,1,https://vk.com/wall-33327655_2215
3,2014_Дорман_Елена_Вячеславовна,Дорман Елена Вячеславовна,2014,2.3,2,3,71,1,https://vk.com/wall-33327655_2216
4,2014_Оськин_Александр_Андреевич,Оськин Александр Андреевич,2014,2.3,2,3,45,1,https://vk.com/wall-33327655_2216
5,2014_Хапкин_Артём_Владимирович,Хапкин Артём Владимирович,2014,4.1,4,1,32,1,https://vk.com/wall-33327655_2240
6,2014_Тулякова_Анна_Владимировна,Тулякова Анна Владимировна,2014,4.1,4,1,19,1,https://vk.com/wall-33327655_2240
7,2014_Матвеев_Дмитрий_Александрович,Матвеев Дмитрий Александрович,2014,4.1,4,1,45,1,https://vk.com/wall-33327655_2240
8,2014_Бунеев_Артём_Валерьевич,Бунеев Артём Валерьевич,2014,4.2,4,2,44,1,https://vk.com/wall-33327655_2241
9,2014_Борисов_Василий_Александрович,Борисов Василий Александрович,2014,4.2,4,2,49,1,https://vk.com/wall-33327655_2241


In [624]:
result_names_df_2.to_csv('ssmm_votes_info_conv4.csv', sep='\t', encoding='utf-8', index=False)

## Creating of final tables

In [625]:
# full table for 4-11 convocations:

def united_table_for_4_11(title_word):
    list_of_df = []
    for i in range(4,11+1):
        df = pd.read_csv('ssmm_'+title_word+'_info_conv'+str(i)+'.csv', sep='\t', encoding='utf-8')
        list_of_df.append(df)
    kek = pd.concat(list_of_df).reset_index().drop(['index'], axis=1)
    kek.to_csv('ssmm_'+title_word+'_info_conv_4_11.csv', sep='\t', encoding='utf-8', index=False)
    return None

title_words = ['announcement','candidates','results','organizers','votes']

for title_word in title_words:
    united_table_for_4_11(title_word)

## results of 2-3 convocations

In [ ]:
# ОСТАЛОСЬ РАСПАРСИТЬ РЕЗУЛЬТАТЫ

In [ ]:
# 2 созыв
# https://vk.com/wall-33327655_725

In [694]:
# conv2_results_dict = vk_post_info_by_link('https://vk.com/wall-33327655_725', 'georgian')
# conv2_results_dict

In [640]:
def stream_by_group(group):
    if group is not None:
        dd = int(group[-2:])
        if dd <= 6:
            stream = 1
        elif dd > 6 and dd <= 12:
            stream = 2
        elif dd >= 21 and dd <= 26:
            stream = 3
        elif dd >= 31:
            stream = 4
    return stream

In [684]:
conv2_result_names = {}

for line in conv2_results_dict['https://vk.com/wall-33327655_725']['list_of_lines']:
    pattern1 = r'([\d]{3})\s([\w\s]+)'
    if re.findall(pattern1, line):
        group_name = re.findall(pattern1, line)[0]
        name = group_name[1]
        academic_group_no = group_name[0]
        year_no = int(academic_group_no[0])
        stream_no = stream_by_group(academic_group_no)
        conv2_result_names[name] = {
            'year_stream_no': str(year_no)+'.'+str(stream_no),
            'year_no': year_no,
            'stream_no': stream_no,
            'academic_group_no': academic_group_no,
#             'convocation_no': 2
        }
    else:
        pattern2 = r'(\w\d)\s([\w\s]+)'
        if re.findall(pattern2, line):
            group_name = re.findall(pattern2, line)[0]
            name = group_name[1]
            academic_group_no = ''
            year_no = int(group_name[0][-1])
            stream_no = 0
            conv2_result_names[name] = {
                'year_stream_no': 'm'+str(year_no),
                'year_no': year_no,
                'stream_no': stream_no,
                'academic_group_no': academic_group_no,
#                 'convocation_no': 2
            }
        
conv2_result_names

{'Остроухова Наталья Владимировна': {'year_stream_no': '1.1',
  'year_no': 1,
  'stream_no': 1,
  'academic_group_no': '101'},
 'Габидов Александр Сергеевич': {'year_stream_no': '1.1',
  'year_no': 1,
  'stream_no': 1,
  'academic_group_no': '104'},
 'Дяченко Мария Игоревна': {'year_stream_no': '1.1',
  'year_no': 1,
  'stream_no': 1,
  'academic_group_no': '104'},
 'Ватутин Кирилл Александрович': {'year_stream_no': '1.3',
  'year_no': 1,
  'stream_no': 3,
  'academic_group_no': '124'},
 'Никифоров Сергей Игоревич': {'year_stream_no': '1.3',
  'year_no': 1,
  'stream_no': 3,
  'academic_group_no': '125'},
 'Матвеев Дмитрий Александрович': {'year_stream_no': '2.1',
  'year_no': 2,
  'stream_no': 1,
  'academic_group_no': '201'},
 'Власов Артем Андреевич': {'year_stream_no': '2.2',
  'year_no': 2,
  'stream_no': 2,
  'academic_group_no': '207'},
 'Курикша Артем Анатольевич': {'year_stream_no': '2.2',
  'year_no': 2,
  'stream_no': 2,
  'academic_group_no': '208'},
 'Тулякова Анна Владими

In [703]:
conv2_result_names_df =  pd.DataFrame(conv2_result_names.items(), columns=['name', 'params'])
conv2_result_names_df_2 = pd.concat([conv2_result_names_df.drop(['params'], axis=1), conv2_result_names_df['params'].apply(pd.Series)], axis=1)
conv2_result_names_df_2 = conv2_result_names_df_2.sort_values(['year_stream_no', 'academic_group_no'],
                  ascending = [True,True]).reset_index().drop(['index'], axis=1)
conv2_result_names_df_2['elected_flg'] = conv2_result_names_df_2.apply(lambda x: 1, axis=1)
conv2_result_names_df_2['convocation_no'] = conv2_result_names_df_2.apply(lambda x: 2, axis=1)
conv2_result_names_df_2['link_date'] = conv2_result_names_df_2.apply(lambda x: conv2_results_dict['https://vk.com/wall-33327655_725']['link_date'], axis=1)
conv2_result_names_df_2['link_time'] = conv2_result_names_df_2.apply(lambda x: conv2_results_dict['https://vk.com/wall-33327655_725']['link_time'], axis=1)
conv2_result_names_df_2['link'] = conv2_result_names_df_2.apply(lambda x: 'https://vk.com/wall-33327655_725', axis=1)

conv2_result_names_df_2


,name,year_stream_no,year_no,stream_no,academic_group_no,elected_flg,convocation_no,link_date,link_time,link
0,Остроухова Наталья Владимировна,1.1,1,1,101,1,2,2012-11-19,16:13,https://vk.com/wall-33327655_725
1,Габидов Александр Сергеевич,1.1,1,1,104,1,2,2012-11-19,16:13,https://vk.com/wall-33327655_725
2,Дяченко Мария Игоревна,1.1,1,1,104,1,2,2012-11-19,16:13,https://vk.com/wall-33327655_725
3,Ватутин Кирилл Александрович,1.3,1,3,124,1,2,2012-11-19,16:13,https://vk.com/wall-33327655_725
4,Никифоров Сергей Игоревич,1.3,1,3,125,1,2,2012-11-19,16:13,https://vk.com/wall-33327655_725
5,Матвеев Дмитрий Александрович,2.1,2,1,201,1,2,2012-11-19,16:13,https://vk.com/wall-33327655_725
6,Власов Артем Андреевич,2.2,2,2,207,1,2,2012-11-19,16:13,https://vk.com/wall-33327655_725
7,Курикша Артем Анатольевич,2.2,2,2,208,1,2,2012-11-19,16:13,https://vk.com/wall-33327655_725
8,Тулякова Анна Владимировна,2.2,2,2,208,1,2,2012-11-19,16:13,https://vk.com/wall-33327655_725
9,Багров Константин Владимирович,2.3,2,3,222,1,2,2012-11-19,16:13,https://vk.com/wall-33327655_725


In [704]:
conv2_result_names_df_2.to_csv('ssmm_votes_info_conv_2.csv', sep='\t', encoding='utf-8', index=False)


In [ ]:
# 3 созыв
# https://vk.com/wall-33327655_1600

In [651]:
conv3_results_dict = vk_post_info_by_link('https://vk.com/wall-33327655_1600', 'georgian')
conv3_results_dict

{'https://vk.com/wall-33327655_1600': {'link_type': 'post',
  'content_type': 'text',
  'content_subtype': 'other',
  'parent_link': '',
  'link_date': datetime.date(2013, 12, 10),
  'link_year_no': 2013,
  'link_month_no': 12,
  'link_day_no': 10,
  'link_date_time': '10 დეკ 2013  1:46',
  'link_time': '1:46',
  'list_of_lines': ['Результаты выборов в Студенческий совет механико-математического факультета МГУ.',
   'Списки приведены пофамильно в алфавитном порядке. Кандидатов, которые выборы не прошли - просим не расстраиваться, а также участвовать в жизни факультета. Ведь помогать в работе Студсовета может каждый!',
   '1 курс',
   '1 поток Алекберов, Бурнус',
   '2 поток Денисов, Суркова, Тюрина',
   '3 поток Ариас_Альварес, Григорьев, Дорман',
   '2 курс',
   '1 поток Дяченко, Локтионов, Остроухова',
   '2 поток Глунчадзе',
   '3 поток Ватутин, Савченко',
   '3 курс',
   '1 поток Андреев, Матвеев',
   '2 поток Борисов, Бунеев, Хамматова',
   '3 поток Быстрицкая, Медведева',
   '4 п

In [688]:
conv3_result_names = {}

list_of_lines = [line for line in conv3_results_dict['https://vk.com/wall-33327655_1600']['list_of_lines'] if 'P.S.' not in line]

for line in list_of_lines:
    pattern1 = r'([\w]+)'
    if 'курс' in line:
        year_no = re.sub('\D', '', line)
    if 'поток' in line:
        stream_no = re.sub('\D', '', line)
        name_list = re.findall(pattern1, line)
        name_list = [w.replace('Ариас_Альварес','Ариас-Альварес') for w in name_list if w != 'поток' and len(w) > 1]
#         print(name_list)
        conv3_result_names[year_no+'.'+stream_no] = {
            'year_no': year_no,
            'stream_no': stream_no,
            'name_list': name_list,
#             'convocation_no': 3
        }
    if 'магистр' in str.lower(line):
        year_no = re.sub('\D', '', line)
        stream_no = 0
        flag = 1
    elif stream_no == 0:
        name_list = re.findall(pattern1, line)
        conv3_result_names['m'+str(year_no)] = {
            'year_no': year_no,
            'stream_no': stream_no,
            'name_list': name_list,
#             'convocation_no': 3
        }

del conv3_result_names['5.4']
conv3_result_names



{'m2': {'year_no': '2', 'stream_no': 0, 'name_list': ['Морозов']},
 'm1': {'year_no': '1', 'stream_no': 0, 'name_list': ['Буряк', 'Короткин']},
 '1.1': {'year_no': '1',
  'stream_no': '1',
  'name_list': ['Алекберов', 'Бурнус']},
 '1.2': {'year_no': '1',
  'stream_no': '2',
  'name_list': ['Денисов', 'Суркова', 'Тюрина']},
 '1.3': {'year_no': '1',
  'stream_no': '3',
  'name_list': ['Ариас-Альварес', 'Григорьев', 'Дорман']},
 '2.1': {'year_no': '2',
  'stream_no': '1',
  'name_list': ['Дяченко', 'Локтионов', 'Остроухова']},
 '2.2': {'year_no': '2', 'stream_no': '2', 'name_list': ['Глунчадзе']},
 '2.3': {'year_no': '2',
  'stream_no': '3',
  'name_list': ['Ватутин', 'Савченко']},
 '3.1': {'year_no': '3',
  'stream_no': '1',
  'name_list': ['Андреев', 'Матвеев']},
 '3.2': {'year_no': '3',
  'stream_no': '2',
  'name_list': ['Борисов', 'Бунеев', 'Хамматова']},
 '3.3': {'year_no': '3',
  'stream_no': '3',
  'name_list': ['Быстрицкая', 'Медведева']},
 '3.4': {'year_no': '3', 'stream_no': '4

In [698]:
conv3_result_names_df =  pd.DataFrame(conv3_result_names.items(), columns=['year_stream_no', 'params'])
conv3_result_names_df_2 = pd.concat([conv3_result_names_df.drop(['params'], axis=1), conv3_result_names_df['params'].apply(pd.Series)], axis=1)
conv3_result_names_df_2 = conv3_result_names_df_2.sort_values(['year_stream_no'],
                  ascending = [True]).reset_index().drop(['index'], axis=1)
conv3_result_names_df_2


,year_stream_no,year_no,stream_no,name_list
0,1.1,1,1,"[Алекберов, Бурнус]"
1,1.2,1,2,"[Денисов, Суркова, Тюрина]"
2,1.3,1,3,"[Ариас-Альварес, Григорьев, Дорман]"
3,2.1,2,1,"[Дяченко, Локтионов, Остроухова]"
4,2.2,2,2,[Глунчадзе]
5,2.3,2,3,"[Ватутин, Савченко]"
6,3.1,3,1,"[Андреев, Матвеев]"
7,3.2,3,2,"[Борисов, Бунеев, Хамматова]"
8,3.3,3,3,"[Быстрицкая, Медведева]"
9,3.4,3,4,[Хасанов]


In [701]:
conv3_result_names_df_3 = conv3_result_names_df_2[['year_stream_no','year_no','stream_no','name_list']].apply(lambda x: x.explode()).reset_index().drop(['index'], axis=1).rename(columns={'name_list':'name'})
conv3_result_names_df_3['elected_flg'] = conv3_result_names_df_3.apply(lambda x: 1, axis=1)
conv3_result_names_df_3['convocation_no'] = conv3_result_names_df_3.apply(lambda x: 3, axis=1)
conv3_result_names_df_3['link_date'] = conv3_result_names_df_3.apply(lambda x: conv3_results_dict['https://vk.com/wall-33327655_1600']['link_date'], axis=1)
conv3_result_names_df_3['link_time'] = conv3_result_names_df_3.apply(lambda x: conv3_results_dict['https://vk.com/wall-33327655_1600']['link_time'], axis=1)
conv3_result_names_df_3['link'] = conv3_result_names_df_3.apply(lambda x: 'https://vk.com/wall-33327655_1600', axis=1)

conv3_result_names_df_3


,year_stream_no,year_no,stream_no,name,elected_flg,convocation_no,link_date,link_time,link
0,1.1,1,1,Алекберов,1,3,2013-12-10,1:46,https://vk.com/wall-33327655_1600
1,1.1,1,1,Бурнус,1,3,2013-12-10,1:46,https://vk.com/wall-33327655_1600
2,1.2,1,2,Денисов,1,3,2013-12-10,1:46,https://vk.com/wall-33327655_1600
3,1.2,1,2,Суркова,1,3,2013-12-10,1:46,https://vk.com/wall-33327655_1600
4,1.2,1,2,Тюрина,1,3,2013-12-10,1:46,https://vk.com/wall-33327655_1600
5,1.3,1,3,Ариас-Альварес,1,3,2013-12-10,1:46,https://vk.com/wall-33327655_1600
6,1.3,1,3,Григорьев,1,3,2013-12-10,1:46,https://vk.com/wall-33327655_1600
7,1.3,1,3,Дорман,1,3,2013-12-10,1:46,https://vk.com/wall-33327655_1600
8,2.1,2,1,Дяченко,1,3,2013-12-10,1:46,https://vk.com/wall-33327655_1600
9,2.1,2,1,Локтионов,1,3,2013-12-10,1:46,https://vk.com/wall-33327655_1600


In [702]:
conv3_result_names_df_3.to_csv('ssmm_votes_info_conv_3.csv', sep='\t', encoding='utf-8', index=False)


## Completeness testing

In [ ]:
# ПОПРИДЖОЙНИВАТЬ АНОНСЫ НА РЕЗУЛЬТАТЫ, ЧТОБЫ ПОНЯТЬ, КАКИЕ ПОСТЫ С РЕЗУЛЬТАТАМИ НЕ РАСПОЗНАЛИСЬ

# - по созывам, внутри созывов:
#     - по потокам
#     - по кандидатам

In [626]:
ssmm_announcements_info = pd.read_csv('ssmm_announcement_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_candidates_info = pd.read_csv('ssmm_candidates_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_results_info = pd.read_csv('ssmm_results_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_organizers_info = pd.read_csv('ssmm_organizers_info_conv_4_11.csv', sep='\t', encoding='utf-8')
ssmm_votes_info = pd.read_csv('ssmm_votes_info_conv_4_11.csv', sep='\t', encoding='utf-8')


In [570]:
# list(ssmm_candidates_info)

In [597]:
# проверка на пустые ячейки
ssmm_candidates_info[ssmm_candidates_info['program_txt'].isna()]


,name,link_year_no_x_name,link_year_no,year_stream_no,year_no,stream_no,academic_group_no,program_txt,link


In [594]:
# ssmm_candidates_info[ssmm_candidates_info['link'] == 'https://vk.com/wall-33327655_8330']
# https://vk.com/wall-33327655_8330

In [565]:
df1 = ssmm_announcements_info[['year_x_year_stream','link']]
df2 = ssmm_results_info[['year_x_year_stream','link']]

df1_x_d2 = df1.merge(df2, left_on='year_x_year_stream', right_on='year_x_year_stream',how='outer')
df1_x_d2_nan = df1_x_d2[df1_x_d2.isna().any(axis=1)]
df1_x_d2_nan
# df1_x_d2[120:180]

# ОСОБЕННОСТИ:
# 2014_m2 - нет в ssmm_announcements_info - не было анонса


,year_x_year_stream,link_x,link_y
164,2014_m2,NaN,https://vk.com/wall-33327655_2313


In [564]:
df1 = ssmm_candidates_info[['link_year_no_x_name','link']]
df2 = ssmm_votes_info[['link_year_no_x_name','link']]

df1_x_d2 = df1.merge(df2, left_on='link_year_no_x_name', right_on='link_year_no_x_name',how='outer')
df1_x_d2_nan = df1_x_d2[df1_x_d2.isna().any(axis=1)]
df1_x_d2_nan


# ОСОБЕННОСТИ (где-то как-то пометить):
# 	2016_Думаревский_Александр_Дмитриевич	https://vk.com/wall-33327655_5144	NaN	 - СНЯЛСЯ
# 	2019_Воронин_Вадим_Юрьевич	https://vk.com/wall-33327655_8323	NaN - СНЯЛСЯ
# 	2014_Вергазов_Марат_Марсович	NaN	https://vk.com/wall-33327655_2313 - НЕ БЫЛО АНОНСА

# ОШИБКИ:
# нет


,link_year_no_x_name,link_x,link_y
131,2016_Думаревский_Александр_Дмитриевич,https://vk.com/wall-33327655_5144,NaN
265,2019_Воронин_Вадим_Юрьевич,https://vk.com/wall-33327655_8323,NaN
390,2014_Вергазов_Марат_Марсович,NaN,https://vk.com/wall-33327655_2313


### finding similar names

In [ ]:
# найти группы имен с минимальным расхождением по расстоянию левенштайна
# найти имена, являющиеся началом других имен (когда забыли про отчество)

In [427]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

string1 = "abc"
string2 = "aef"

print(levenshtein(string1, string2))

In [627]:
names = list(ssmm_candidates_info['name']) + list(ssmm_votes_info['name'])

from collections import Counter

names_stat = Counter(names) #.most_common()
# names_stat

# names_uniq = list(set(names))
# names_uniq

In [618]:
# выявляем похожие имена, чтобы потом заменить РУКАМИ

for a in names_stat.keys():
    for b in names_stat.keys():
        if a != b and levenshtein(a, b) <= 4 and names_stat[a] >= names_stat[b]:
            print(levenshtein(a, b), ' - ', a, names_stat[a], ' : ', names_stat[b], b)

In [628]:
# выявляем имена без отчества, чтобы потом заменить РУКАМИ

for a in names_stat.keys():
    for b in names_stat.keys():
        if len(a) > len(b) and levenshtein(a[:len(b)], b) <= 4:
            print(a, names_stat[a], ' : ', names_stat[b], b)


In [629]:
# выявляем имена с перепутанными отчествами, чтобы потом заменить РУКАМИ

for a in names_stat.keys():
    for b in names_stat.keys():
        if a != b and levenshtein(a[:12], b[:12]) <= 2 and names_stat[a] >= names_stat[b]:
            print(a, names_stat[a], ' : ', names_stat[b], b)

# Сакаев Роберт Марселевич 5  :  1 Сакаев Роберт Викторович


Порфирьева Наталия Алексеевна 2  :  2 Перфильева Нелли Андреевна
Федотов Константин Алексеевич 2  :  2 Фёдоров Константин Дмитриевич
Алиев Рамиль Камиль Оглы 2  :  2 Алеев Дамиль Павлович
Перфильева Нелли Андреевна 2  :  2 Порфирьева Наталия Алексеевна
Фёдоров Константин Дмитриевич 2  :  2 Федотов Константин Алексеевич
Третьяков Пётр Максимович 6  :  4 Третьяков Артём Алексеевич
Лобанова Алёна Валерьевна 6  :  2 Лобанова Анастасия Алексеевна
Алеев Дамиль Павлович 2  :  2 Алиев Рамиль Камиль Оглы


In [463]:
# incorrect: correct
correct_names_dict = {
#     из-за Е/Ё:
    'Муравьева Анна Сергеевна': 'Муравьёва Анна Сергеевна',
    'Третьяков Артем Алексеевич': 'Третьяков Артём Алексеевич',
    'Рогачев Александр Викторович': 'Рогачёв Александр Викторович',
    'Третьяков Петр Максимович': 'Третьяков Пётр Максимович',
    'Пушкарева Анна Константиновна': 'Пушкарёва Анна Константиновна',
#     из-за других букв:
    'Порфирьева Наталья Алексеевна': 'Порфирьева Наталия Алексеевна',
    'Лобанова Елена Валерьевна': 'Лобанова Алёна Валерьевна',
    'Иллизаров Год Эмануилович': 'Илизаров Год Эмануилович',
    'Гусейнов Сеймур Элданизович': 'Гусейнов Сэймур Элданизович',
#     из-за пропущенного отчества:
    'Лапина София': 'Лапина Софья Михайловна',   
#     из-за перепутанного отчества:
    'Сакаев Роберт Викторович': 'Сакаев Роберт Марселевич',   
}
# correct_names_dict

correct_names_df =  pd.DataFrame(correct_names_dict.items(), columns=['incorrect_name', 'correct_name'])
correct_names_df

,incorrect_name,correct_name
0,Муравьева Анна Сергеевна,Муравьёва Анна Сергеевна
1,Третьяков Артем Алексеевич,Третьяков Артём Алексеевич
2,Рогачев Александр Викторович,Рогачёв Александр Викторович
3,Третьяков Петр Максимович,Третьяков Пётр Максимович
4,Пушкарева Анна Константиновна,Пушкарёва Анна Константиновна
5,Порфирьева Наталья Алексеевна,Порфирьева Наталия Алексеевна
6,Лобанова Елена Валерьевна,Лобанова Алёна Валерьевна
7,Иллизаров Год Эмануилович,Илизаров Год Эмануилович
8,Гусейнов Сеймур Элданизович,Гусейнов Сэймур Элданизович
9,Лапина София,Лапина Софья Михайловна
